## Preprocessing

In [12]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pandas as pd
import tensorflow as tf
import operator

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [13]:
# Install keras-tuner for hyperparamter tuning later on
pip install keras-tuner

SyntaxError: ignored

In [15]:
# # Drop the non-beneficial ID columns, 'EIN', 'NAME', 'STATUS', and 'SPECIAL_CONSIDERATIONS'.
df = application_df.drop(columns=["EIN","NAME","STATUS","SPECIAL_CONSIDERATIONS"])
df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,5000,0
34297,T5,Independent,C3000,ProductDev,Association,0,5000,1


In [16]:
# Verify that the number of columns has changed after the drop
# Original data
print(len(application_df.columns))

# dropped-column data
print(len(df.columns))

12
8


In [17]:
# Determine the number of unique values in each column.
df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [18]:
# Look at APPLICATION_TYPE value counts for binning

print(df["APPLICATION_TYPE"].value_counts())

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [19]:
# Create function to fee dataframe, column name, and other parameters to determine
  # how much of the data (in percentage) is being cut off at different cutoff values.
def find_df_percentage(dataframe, column, operator_keep, operator_cutoff, cutoff):


  # Condense variables
  df = dataframe
  cutoff_value = cutoff

  # Confirming info when using the function later
  print(f"The '{column}' column will be filtered by {cutoff}")
  print("")

  # Create two lists of application types and their value_counts (frequency) in the dataframe
  # One list for those whose value is above cutoff
  # One list for those whose value is below cutoff

  application_types_keep = [(index, value) for index, value in df[column].value_counts().items() if operator_keep(value, cutoff_value)]
  application_types_cutoff = [(index, value) for index, value in df[column].value_counts().items() if operator_cutoff(value, cutoff_value)]

  # Get the sums of rows kept and cutoff, then the total of the sums.
  data_rows_kept = sum([value for index, value in application_types_keep])
  data_rows_cutoff = sum([value for index, value in application_types_cutoff])
  data_rows_total = data_rows_kept + data_rows_cutoff

  # Test if the total rows == the dataset they're filtering to verify no data loss
  total_check = data_rows_total == len(df)
  print(f"It is {total_check} that there was no data lost when filtering the dataframe.")
  print("")

  # Calculate percentage of part/whole
  def calculate_percentage(part, whole):
    return (part / whole) * 100

  # Calculate the percentage of each of the sums to the entire dataset.
  percentage_of_kept_data = calculate_percentage(data_rows_kept, data_rows_total)
  percentage_of_cutoff_data = calculate_percentage(data_rows_cutoff, data_rows_total)

  # Print the information
  print(f"{data_rows_kept} rows of the data were kept, which is about {round(percentage_of_kept_data, 2)} of the data full dataset.")
  print(f"{data_rows_cutoff} rows of the data were cutoff, which is about {round(percentage_of_cutoff_data, 2)} of the data full dataset.")

  # Return raw percentage (float) values
  return percentage_of_kept_data, percentage_of_cutoff_data

# Application_type value_counts

In [20]:
# Calling function to estimate the percentage of the original dataset that a cutoff_value of 50 would remove

# Operators initialized into variables for better readibility
greater_than_equal = operator.ge
less_than = operator.lt
cutoff_value = 500

# Run function designed above
perc_kept, perc_cutoff = find_df_percentage(dataframe= df,
                   column= "APPLICATION_TYPE",
                   operator_keep= greater_than_equal,
                   operator_cutoff= less_than,
                   cutoff= cutoff_value)

# Using the floats returned by the function
print("")
print("Raw Percentages:")
print(f"{perc_kept} kept")
print(f"{perc_cutoff} cutoff")

The 'APPLICATION_TYPE' column will be filtered by 500

It is True that there was no data lost when filtering the dataframe.

34023 rows of the data were kept, which is about 99.2 of the data full dataset.
276 rows of the data were cutoff, which is about 0.8 of the data full dataset.

Raw Percentages:
99.19531181667104 kept
0.8046881833289601 cutoff


In [21]:
# Choose a cutoff value and create a list of application types to be replaced and use the variable name `application_types_to_replace`

# Value chosen as 500 (verified in cell above)
# list comprehension below selects only the application types
application_types_cutoff = [(index, value) for index, value in df['APPLICATION_TYPE'].value_counts().items() if value < 500]
application_types_to_replace = [index for index, value in application_types_cutoff]

# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

# Classification value_counts

In [22]:
# Look at CLASSIFICATION value counts for binning
print(df["CLASSIFICATION"].value_counts()[:10])

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
Name: CLASSIFICATION, dtype: int64


In [23]:
# Calling function to estimate the percentage of the original dataset that a cutoff_value of 50 would remove

# Operators initialized into variables for better readibility
greater_than = operator.gt
less_than_equal = operator.le
cutoff_value = 1000

# Run function
perc_kept, perc_cutoff = find_df_percentage(dataframe= df,
                   column= "CLASSIFICATION",
                   operator_keep= greater_than,
                   operator_cutoff= less_than_equal,
                   cutoff= cutoff_value)

print("")
print("Raw Percentages:")
print(f"{perc_kept} kept")
print(f"{perc_cutoff} cutoff")

The 'CLASSIFICATION' column will be filtered by 1000

It is True that there was no data lost when filtering the dataframe.

32038 rows of the data were kept, which is about 93.41 of the data full dataset.
2261 rows of the data were cutoff, which is about 6.59 of the data full dataset.

Raw Percentages:
93.40797107787398 kept
6.59202892212601 cutoff


In [24]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classification_types_to_replace`

# Value chosen as 1000 (verified in cell above)
# list comprehension below selects only the application types
cutoff_value = 1000
classification_types_cutoff = [(index, value) for index, value in df["CLASSIFICATION"].value_counts().items() if value < cutoff_value]
classification_types_to_replace = [index for index, value in classification_types_cutoff]

# Replace in dataframe
for cls in classification_types_to_replace:
    df["CLASSIFICATION"] = df["CLASSIFICATION"].replace(cls,"Other")

# Check to make sure binning was successful
df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [25]:
# Convert categorical data to numeric with `pd.get_dummies`
df_encoded = pd.get_dummies(df, drop_first=True)
df_encoded.head()


,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,108590,1,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6692,1,0,0,1,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
4,142590,1,0,0,1,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0


In [26]:
X = df_encoded.drop(columns="IS_SUCCESSFUL").values
y = df_encoded["IS_SUCCESSFUL"].values


In [27]:
# Instantiate the PCA instance with 3 PCA variables
pca = PCA(n_components=3)

# Fit the PCA model on the transformed credit card DataFrame
pca_X = pca.fit_transform(df_encoded)

# Review the first 5 rows of list data
pca_X[:5]

array([[-2.76419868e+06,  1.11442111e+00,  1.08266257e+00],
       [-2.66060868e+06, -2.36184750e-01,  2.66675083e-01],
       [-2.76419868e+06,  1.91597925e+00,  6.92363039e-02],
       [-2.76250668e+06, -2.87489593e-01, -1.01996665e-01],
       [-2.62660868e+06, -2.60001711e-01,  7.74176315e-01]])

In [28]:
# Calculate the PCA explained variance ratio
pca.explained_variance_ratio_

array([1.00000000e+00, 7.17469969e-17, 4.56765323e-17])

In [29]:
# Create a dataframe using the PCA data and target values
pca_X_df = pd.DataFrame(
    pca_X,
    columns=["PCA1", "PCA2", "PCA3"]
)
pca_X_df["IS_SUCCESSFUL"] = y
pca_X_df.head()

,PCA1,PCA2,PCA3,IS_SUCCESSFUL
0,-2.764199e+06,1.114421,1.082663,1
1,-2.660609e+06,-0.236185,0.266675,1
2,-2.764199e+06,1.915979,0.069236,0
3,-2.762507e+06,-0.287490,-0.101997,1
4,-2.626609e+06,-0.260002,0.774176,1


In [30]:
# Split our preprocessed data into our features and target arrays again

X = pca_X_df.drop(columns="IS_SUCCESSFUL").values
y = pca_X_df["IS_SUCCESSFUL"].values

# Shape targets
y = y.reshape(-1, 1)


# # Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1)


In [31]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Determine best parameters using Hyper Parameter Tuning

In [ ]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid','selu',"elu"])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=3))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [ ]:
# Import the kerastuner library
import keras_tuner as kt

# Create tuner and establish parameters
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

Reloading Tuner from ./untitled_project/tuner0.json


In [ ]:
# WARNING: THIS WILL TAKE A LONG TIME. LAST RUN WAS 44 MINUTES.
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 56 Complete [00h 01m 30s]
val_accuracy: 0.9560349583625793

Best val_accuracy So Far: 0.9813411235809326
Total elapsed time: 00h 44m 08s


In [ ]:
# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'relu', 'first_units': 5, 'num_layers': 2, 'units_0': 3, 'units_1': 7, 'units_2': 7, 'units_3': 7, 'units_4': 3, 'units_5': 1, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0047'}
{'activation': 'tanh', 'first_units': 7, 'num_layers': 4, 'units_0': 7, 'units_1': 9, 'units_2': 9, 'units_3': 7, 'units_4': 1, 'units_5': 3, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0040'}
{'activation': 'relu', 'first_units': 7, 'num_layers': 6, 'units_0': 7, 'units_1': 5, 'units_2': 5, 'units_3': 9, 'units_4': 3, 'tuner/epochs': 7, 'tuner/initial_epoch': 3, 'tuner/bracket': 2, 'tuner/round': 1, 'units_5': 1, 'tuner/trial_id': '0003'}


In [ ]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.0780 - accuracy: 0.9813 - 853ms/epoch - 3ms/step
Loss: 0.07797562330961227, Accuracy: 0.9813411235809326
268/268 - 1s - loss: 0.0483 - accuracy: 0.9795 - 942ms/epoch - 4ms/step
Loss: 0.04833395034074783, Accuracy: 0.979475200176239
268/268 - 1s - loss: 0.1216 - accuracy: 0.9733 - 1s/epoch - 4ms/step
Loss: 0.12161662429571152, Accuracy: 0.9732944369316101


In [ ]:
second_best = tuner.get_best_models(2)
second_best

NameError: ignored

# Compile, Train and Evaluate the Model

In [32]:
# Top 3 models according the the tuner
# Model #1
# {'activation': 'relu', 'first_units': 5, 'num_layers': 2, 'units_0': 3, 'units_1': 7, 'units_2': 7, 'units_3': 7, 'units_4': 3, 'units_5': 1, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0047'}

# Model #2 (Best fit, given loss: 0.0483 - accuracy: 0.9795. Loss is 3% lower than model #1, which performed negligably better.)
# {'activation': 'tanh', 'first_units': 7, 'num_layers': 4, 'units_0': 7, 'units_1': 9, 'units_2': 9, 'units_3': 7, 'units_4': 1, 'units_5': 3, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0040'}

# Model #3
# {'activation': 'relu', 'first_units': 7, 'num_layers': 6, 'units_0': 7, 'units_1': 5, 'units_2': 5, 'units_3': 9, 'units_4': 3, 'tuner/epochs': 7, 'tuner/initial_epoch': 3, 'tuner/bracket': 2, 'tuner/round': 1, 'units_5': 1, 'tuner/trial_id': '0003'}

In [33]:
# Define the model based on Model #2 above
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=7, activation="tanh", input_dim=3))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=7, activation="tanh"))

nn.add(tf.keras.layers.Dense(units=9, activation="tanh"))

nn.add(tf.keras.layers.Dense(units=9, activation="tanh"))

nn.add(tf.keras.layers.Dense(units=7, activation="tanh"))

nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))

nn.add(tf.keras.layers.Dense(units=3, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 7)                 28        
                                                                 
 dense_1 (Dense)             (None, 7)                 56        
                                                                 
 dense_2 (Dense)             (None, 9)                 72        
                                                                 
 dense_3 (Dense)             (None, 9)                 90        
                                                                 
 dense_4 (Dense)             (None, 7)                 70        
                                                                 
 dense_5 (Dense)             (None, 1)                 8         
                                                                 
 dense_6 (Dense)             (None, 3)                 6

In [34]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [35]:
# Train the model
# import the ModelCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath='model_weights.h5',  # Filepath to save the model weights
    save_weights_only=True,  # Only save the model weights, not the entire model
    save_best_only=False,  # Save the weights at every five epochs
    period=5,  # Save weights every five epochs
    verbose=1  # Display a message when the weights are saved
)


fit_nn = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 11s 4ms/step - loss: 0.3709 - accuracy: 0.9135
Epoch 2/50
804/804 [==============================] - 3s 4ms/step - loss: 0.1864 - accuracy: 0.9561
Epoch 3/50
804/804 [==============================] - 5s 6ms/step - loss: 0.1775 - accuracy: 0.9560
Epoch 4/50
804/804 [==============================] - 3s 4ms/step - loss: 0.1641 - accuracy: 0.9530
Epoch 5/50
804/804 [==============================] - 3s 4ms/step - loss: 0.1499 - accuracy: 0.9548
Epoch 6/50
804/804 [==============================] - 4s 4ms/step - loss: 0.1454 - accuracy: 0.9551
Epoch 7/50
804/804 [==============================] - 7s 8ms/step - loss: 0.1415 - accuracy: 0.9559
Epoch 8/50
804/804 [==============================] - 7s 8ms/step - loss: 0.1346 - accuracy: 0.9548
Epoch 9/50
804/804 [==============================] - 8s 10ms/step - loss: 0.1278 - accuracy: 0.9549
Epoch 10/50
804/804 [==============================] - 6s 7ms/step - loss: 0.1216 - accuracy: 0.95

In [37]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.0324 - accuracy: 0.9889 - 1s/epoch - 5ms/step
Loss: 0.032439202070236206, Accuracy: 0.9889212846755981


In [38]:
# Export the model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
